# Exemples

## Matrice de Hilbert

Une matrice de Hilbert est une matrice carrée de terme général :

$$ H_{ij} = \frac{1}{i+j-1} $$

La matrice de Hilbert de taille 6 s'écrit :

$$\begin{pmatrix}
 1   & 1/2 & 1/3 & 1/4 & 1/5  & 1/6  \\
 1/2 & 1/3 & 1/4 & 1/5 & 1/6  & 1/7  \\
 1/3 & 1/4 & 1/5 & 1/6 & 1/7  & 1/8  \\
 1/4 & 1/5 & 1/6 & 1/7 & 1/8  & 1/9  \\
 1/5 & 1/6 & 1/7 & 1/8 & 1/9  & 1/10 \\
 1/6 & 1/7 & 1/8 & 1/9 & 1/10 & 1/11  
\end{pmatrix}$$

On initialise une matrice de Hilbert de taille $n$ avec des fractions fractionnelles de sorte de faire calculs exacts.

In [3]:
n = 10
A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
print(A.str(unicode=True))

⎛   1  1/2  1/3  1/4  1/5  1/6  1/7  1/8  1/9 1/10⎞
⎜ 1/2  1/3  1/4  1/5  1/6  1/7  1/8  1/9 1/10 1/11⎟
⎜ 1/3  1/4  1/5  1/6  1/7  1/8  1/9 1/10 1/11 1/12⎟
⎜ 1/4  1/5  1/6  1/7  1/8  1/9 1/10 1/11 1/12 1/13⎟
⎜ 1/5  1/6  1/7  1/8  1/9 1/10 1/11 1/12 1/13 1/14⎟
⎜ 1/6  1/7  1/8  1/9 1/10 1/11 1/12 1/13 1/14 1/15⎟
⎜ 1/7  1/8  1/9 1/10 1/11 1/12 1/13 1/14 1/15 1/16⎟
⎜ 1/8  1/9 1/10 1/11 1/12 1/13 1/14 1/15 1/16 1/17⎟
⎜ 1/9 1/10 1/11 1/12 1/13 1/14 1/15 1/16 1/17 1/18⎟
⎝1/10 1/11 1/12 1/13 1/14 1/15 1/16 1/17 1/18 1/19⎠


On choisit un second membre $b$ tel que chaque composante du vecteur solution $x$ vaut $1$ :

In [4]:
x = vector(QQ, [1 for i in range(0,n)])
b = A*x
print(b.column().str(unicode=True))

⎛        7381/2520⎞
⎜      55991/27720⎟
⎜      44441/27720⎟
⎜    485333/360360⎟
⎜    420983/360360⎟
⎜      74587/72072⎟
⎜    134159/144144⎟
⎜  2074783/2450448⎟
⎜    634871/816816⎟
⎝33464927/46558512⎠


On vérifie la solution :

In [5]:
x = A\b
print(x.column().str(unicode=True))

⎛1⎞
⎜1⎟
⎜1⎟
⎜1⎟
⎜1⎟
⎜1⎟
⎜1⎟
⎜1⎟
⎜1⎟
⎝1⎠


On perturbe la dernière composante du second membre de (1/1000000), on résoud le système et on s'attend à trouver une solution proche du vecteur dont les composantes valents $1$ :

In [6]:
b[n-1] *= (1+1/(1000000)) #  perturbation du second membre
x_pert = A\b
print(x_pert.column().str(unicode=True))

⎛  16935073/50400000⎞
⎜    34024927/560000⎟
⎜  -367834197/280000⎟
⎜    368144197/30000⎟
⎜  -4785404561/80000⎟
⎜ 33498591927/200000⎟
⎜-33498271927/120000⎟
⎜   4785502061/17500⎟
⎜-81352677537/560000⎟
⎝81355757537/2520000⎠


In [7]:
err = max(abs(float(x[i]-x_pert[i])) for i in range(0,n))
print("Norme infinie de l'écart entre la solution et la solution perturbée :", err)

Norme infinie de l'écart entre la solution et la solution perturbée : 279153.26605833333


## Exemple de Forsythe

On considère le système :

$$
\begin{pmatrix}
 0.0001   & 1 \\
 1   & 1 
\end{pmatrix}
\begin{pmatrix}
 x_1  \\
 x_2  
\end{pmatrix}
=
\begin{pmatrix}
 1.0001  \\
 2  
\end{pmatrix}
$$

pour lequel la solution exacte est $x_1 = 1$ et $x_2 = 1$.

On utilise l'algorithme de l’élimination de Gauss pour des flottants avec 3 chiffres significatifs (en base 10).

In [44]:
import numpy as np
from mpmath import mp

def gaussian_elimination_without_pivoting(a, b): 
    n = b.size
    for i in range(n-1):
        # elimination
        li = a[i+1:,i]/a[i,i]
        b[i+1:] = b[i+1:] - li * b[i]
        a[i+1:] = a[i+1:] - li.reshape(n-i-1,1)*a[i]
        
def backward_substitution(a, b):
    n = b.size
    x = b[0]*np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.sum(a[i,i+1:]*x[i+1:])) / a[i,i]
    return x

def gauss_solve_without_pivoting(a, b):
    ag = np.copy(a) 
    bg = np.copy(b)
    gaussian_elimination_without_pivoting(ag, bg)
    return backward_substitution(ag, bg)

mp.dps = 3
a11 = mp.mpf('1e-4')
b1 = mp.mpf('1.0001')

one = mp.mpf('1')
two = mp.mpf('2')

a = np.matrix([[a11, one], [one, one]])
b = np.array([b1, two])

x = gauss_solve_without_pivoting(a, b)
print("Solution : x1 =", x[0], "et x2 =", x[1])

Solution : x1 = 2.44 et x2 = 1.0


On intervertit les deux lignes du système :

$$
\begin{pmatrix}
 1   & 1 \\ 
 0.0001   & 1 
\end{pmatrix}
\begin{pmatrix}
 x_2  \\
 x_1  
\end{pmatrix}
=
\begin{pmatrix}
  2  \\
  1.0001
\end{pmatrix}
$$

In [46]:
mp.dps = 3
a21 = mp.mpf('1e-4')
b1 = mp.mpf('1.0001')

one = mp.mpf('1')
two = mp.mpf('2')

a = np.matrix([[one, one],[a21, one]])
b = np.array([two, b1])

x = gauss_solve_without_pivoting(a, b)
print("Solution : x1 =", x[1], "et x2 =", x[0])

Solution : x1 = 1.0 et x2 = 1.0
